This notebook will generate the detection file for deep sort to evaluate the tracker and feature extraction performance.
So we can have the idea where can go wrong in the tracker

In [12]:
import os
import pandas as pd
import glob
import numpy as np

In [2]:
KITTI_LABEL = ["frame", "track_id", "class_name", "truncated",
                 "occluded", "alpha", "bbox_l", "bbox_t",
                 "bbox_r", "bbox_b", "hdim", "wdim",
                 "ldim", "locx", "locy", "locz", "rot_y"]

In [3]:
CLASS_NAME = ['Car', 'Cyclist', 'Misc', 'Pedestrian', 'Person', 'Tram', 'Truck','Van']

In [14]:
with open("data/kitti.names", 'w') as f:
    for i in CLASS_NAME:
        f.writelines(i + os.linesep)

In [5]:
CLASS_DICT = {k:v for v, k in enumerate(CLASS_NAME)}

In [6]:
CLASS_DICT

{'Car': 0,
 'Cyclist': 1,
 'Misc': 2,
 'Pedestrian': 3,
 'Person': 4,
 'Tram': 5,
 'Truck': 6,
 'Van': 7}

In [7]:
LABEL_PATH = r"/datasets/kitti_tracking/label/"

In [8]:
!ls $LABEL_PATH

0000.txt  0003.txt  0006.txt  0009.txt	0012.txt  0015.txt  0018.txt
0001.txt  0004.txt  0007.txt  0010.txt	0013.txt  0016.txt  0019.txt
0002.txt  0005.txt  0008.txt  0011.txt	0014.txt  0017.txt  0020.txt


In [9]:
labels = glob.glob(LABEL_PATH + "*.txt")

In [10]:
EXPORT_DIR = '../results/kitti_track_02_gt/raw_detections'
# EXPORT_DIR_2 = '../results/kitti_track_03_gt/raw_detections'
os.makedirs(EXPORT_DIR, exist_ok=True)
# os.makedirs(EXPORT_DIR_2, exist_ok=True)

In [13]:
class_ids = []
for label in labels:
#     print(label)
    name, ext = os.path.splitext(os.path.basename(label))
    df = pd.read_csv(label, names= KITTI_LABEL, delimiter=" ")
    df = df[(df[KITTI_LABEL[2]] != "DontCare")]
    df['class_id'] = df['class_name'].map(CLASS_DICT)
    df['w'] = df['bbox_r'] - df['bbox_l']
    df['h'] = df['bbox_b'] - df['bbox_t']
    export_label_data = df[['frame', 'track_id', 'bbox_l', 'bbox_t', 'w', 'h', 'truncated', 'class_id', 'occluded', 'wdim']]
    export_label_data = export_label_data.copy(deep=True)
    export_label_data['truncated'] = 1
    export_label_data['track_id'] = -1
    export_label_data['occluded'] = -1
    export_label_data['wdim'] = -1
    export_numpy = export_label_data.to_numpy()
    output_filename = os.path.join(EXPORT_DIR, "%s.npy" % name)
    output_txtname = os.path.join(EXPORT_DIR, "%s.txt" % name)
    np.save(output_filename, export_numpy, allow_pickle=False)
    np.savetxt(output_txtname, export_numpy, fmt='%4.2f')